In [18]:
import pandas as pd
from numpy import loadtxt
from matplotlib import pyplot  as plt
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from google.colab import drive
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import os
from google.colab import drive
from sklearn.linear_model import LogisticRegression, LinearRegression
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
tensorflow.__version__

'2.15.0'

In [19]:
# GIT VERSIONING CONTROL
drive.mount('/content/drive/')
%cd /content/drive/MyDrive/CAUSALITY/causal_fairness/

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/MyDrive/CAUSALITY/causal_fairness


In [21]:
# READ GANITE'S OUTPUT
X_train = np.loadtxt("/content/drive/My Drive/CAUSALITY/causal_fairness/DATA/train_x.csv", delimiter=",",skiprows=1)
train_t = np.loadtxt("/content/drive/My Drive/CAUSALITY/causal_fairness/DATA/train_t.csv", delimiter=",",skiprows=1)
y_train = np.loadtxt("/content/drive/My Drive/CAUSALITY/causal_fairness/DATA/train_y.csv", delimiter=",",skiprows=1)
train_potential_y = np.loadtxt("/content/drive/My Drive/CAUSALITY/causal_fairness/DATA/train_potential_y.csv", delimiter=",",skiprows=1)
X_test = np.loadtxt("/content/drive/My Drive/CAUSALITY/causal_fairness/DATA/test_x.csv", delimiter=",",skiprows=1)
test_t = np.loadtxt("/content/drive/My Drive/CAUSALITY/causal_fairness/DATA/test_t.csv", delimiter=",",skiprows=1)
test_potential_y = np.loadtxt("/content/drive/My Drive/CAUSALITY/causal_fairness/DATA/test_potential_y.csv", delimiter=",",skiprows=1)
test_y_hat = np.loadtxt("/content/drive/My Drive/CAUSALITY/causal_fairness/DATA/test_potest_y_hat.csv", delimiter=",",skiprows=1)

In [23]:
# Extract counterfactuals
train_cf=[]
for i in range(len(train_t)):
  train_cf.append(train_potential_y[i][1-int(train_t[i])])
test_cf=[]
for i in range(len(test_t)):
  test_cf.append(test_y_hat[i][1-int(test_t[i])])
test_cf = [round(x) for x in test_cf]

In [44]:
def doubly_robust(df, X, T, Y):
    ps = LogisticRegression(C=1e6, max_iter=100000).fit(df[X], df[T]).predict_proba(df[X])[:, 1]
    mu0 = LinearRegression().fit(df.query(f"{T}==0")[X], df.query(f"{T}==0")[Y]).predict(df[X])
    mu1 = LinearRegression().fit(df.query(f"{T}==1")[X], df.query(f"{T}==1")[Y]).predict(df[X])
    return (
        np.mean(df[T]*(df[Y] - mu1)/ps + mu1) -
        np.mean((1-df[T])*(df[Y] - mu0)/(1-ps) + mu0)
    )

In [45]:
model = Sequential()
model.add(Dense(10, input_dim=30, activation='sigmoid'))
model.add(Dense(10, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))

In [46]:
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

In [100]:
def metrics_calculator(reference):
  history=model.fit(X_train, reference, epochs=50, batch_size=25)
  loss, accuracy = model.evaluate(X_train, reference)
  X_test_df=pd.DataFrame(X_test)
  treatment_df=pd.DataFrame(test_t, columns=["Treatment"])
  #y_test_df=pd.DataFrame(y_test,columns=["Output"]) Uncomment if want to check with Y_test
  prediccion = model.predict(X_test)
  #prediccion = [round(x[0]) for x in prediccion] Uncomment if want to use classes.  Using probs now
  predictions_df=pd.DataFrame(prediccion,columns=["Output"])
  T = 'Treatment'
  Y = 'Output'
  X = X_test_df.columns
  #full_test_df=pd.concat([X_test_df, treatment_df, y_test_df], axis=1) Uncomment if want to check with Y_test
  full_test_df=pd.concat([X_test_df, treatment_df, predictions_df], axis=1)
  full_test_df["Treatment"]=full_test_df["Treatment"].astype('bool')
  ate=doubly_robust(full_test_df, X, T, Y)
  return [accuracy, loss, ate]

In [102]:
accst1=[]
atest1=[]
lossest1=[]

accst2=[]
atest2=[]
lossest2=[]


accst3=[]
atest3=[]
lossest3=[]

# Set lambda
lam=0.2

for i in range(3):
  # No regularization
  rest1=metrics_calculator((y_train))
  accst1.append(rest1[0])
  lossest1.append(rest1[1])
  atest1.append(rest1[2])
  # First regularization
  rest2=metrics_calculator((y_train+train_cf)/2)
  accst2.append(rest2[0])
  lossest2.append(rest2[1])
  atest2.append(rest2[2])
  # Second regularization
  rest3=metrics_calculator(lam*y_train+(1-lam)*np.array(train_cf))
  accst3.append(rest3[0])
  lossest3.append(rest3[1])
  atest3.append(rest3[2])

Epoch 1/50
365/365 [==============================] - 2s 4ms/step - loss: 0.0853 - accuracy: 0.8894
Epoch 2/50
365/365 [==============================] - 1s 4ms/step - loss: 0.0852 - accuracy: 0.8885
Epoch 3/50
365/365 [==============================] - 2s 6ms/step - loss: 0.0853 - accuracy: 0.8893
Epoch 4/50
365/365 [==============================] - 2s 7ms/step - loss: 0.0852 - accuracy: 0.8883
Epoch 5/50
365/365 [==============================] - 1s 4ms/step - loss: 0.0854 - accuracy: 0.8874
Epoch 6/50
365/365 [==============================] - 1s 3ms/step - loss: 0.0856 - accuracy: 0.8878
Epoch 7/50
365/365 [==============================] - 1s 2ms/step - loss: 0.0851 - accuracy: 0.8889
Epoch 8/50
365/365 [==============================] - 1s 2ms/step - loss: 0.0852 - accuracy: 0.8883
Epoch 9/50
365/365 [==============================] - 1s 2ms/step - loss: 0.0852 - accuracy: 0.8898
Epoch 10/50
365/365 [==============================] - 1s 2ms/step - loss: 0.0854 - accuracy: 0.8887

In [103]:
li = [accst1, lossest1, atest1]
No_reg=pd.DataFrame(data = li).T
No_reg.columns=["Accuracy No reg","Loss No reg", "ATE no reg" ]

In [104]:
li = [accst2, lossest2, atest2]
reg_1=pd.DataFrame(data = li).T
reg_1.columns=["Accuracy Reg 1","Loss Reg 1", "ATE Reg 1" ]

In [105]:
li = [accst3, lossest3, atest3]
reg_2=pd.DataFrame(data = li).T
reg_2.columns=["Accuracy Reg 2","Loss Reg 2", "ATE Reg 2" ]

In [106]:
reg_1

,Accuracy Reg 1,Loss Reg 1,ATE Reg 1
0,0.836075,0.059804,0.003343
1,0.840461,0.057279,0.001699
2,0.840899,0.057080,0.001934


In [107]:
reg_2

,Accuracy Reg 2,Loss Reg 2,ATE Reg 2
0,0.840570,0.066769,0.001989
1,0.839474,0.066460,0.001047
2,0.840789,0.066624,0.001061


In [108]:
No_reg

,Accuracy No reg,Loss No reg,ATE no reg
0,0.893092,0.083383,0.002194
1,0.893421,0.083376,0.001960
2,0.893640,0.082145,0.000896
